# mounting the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')   

Mounted at /content/drive


In [ ]:
s1 = "I have eaten rice"
s2 = "I have not eaten rice"

In [ ]:
# some preprocessing
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-' :
            continue
        result.append(token.lemma_)
    return " ".join(result)

# cosine similarity

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def cosine_similarity(s1,s2):

  X_list = word_tokenize(s1) 
  Y_list = word_tokenize(s2)
    
  # sw contains the list of stopwords
  sw = stopwords.words('english') 
  l1 =[];l2 =[]
    
  # remove stop words from the string
  X_set = {w for w in X_list if not w in sw} 
  Y_set = {w for w in Y_list if not w in sw}
    
  # form a set containing keywords of both strings 
  rvector = X_set.union(Y_set) 
  
  for w in rvector:
      if w in X_set: l1.append(1) 
      else: l1.append(0)
      if w in Y_set: l2.append(1)
      else: l2.append(0)
  c = 0
    
  # cosine formula 
  for i in range(len(rvector)):
        c+= l1[i]*l2[i]

  s1 = 0
  s2 = 0

  for i in range(0,len(l1)):
    s1 = s1 + l1[i]

  for i in range(0,len(l2)):
    s2 = s2 + l2[i]

  p = float((float(s1 * s2 ))**0.5)
  cosine = c/p;

  return float(cosine)


In [ ]:
cosine_similarity(s1,s2)

1.0

# dice similarity

In [ ]:
def dice_coefficient(a, b):
    a = process_text(a)
    b = process_text(b)
    a_bigrams = set(a)
    b_bigrams = set(b)
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0/(len(a_bigrams) + len(b_bigrams))

In [ ]:
dice_coefficient(s1,s2)

1.0

# bert similarity

In [ ]:
!pip install bert-score

     |████████████████████████████████| 60 kB 3.4 MB/s 
     |████████████████████████████████| 3.4 MB 10.7 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
     |████████████████████████████████| 3.3 MB 42.7 MB/s 
     |████████████████████████████████| 61 kB 485 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import re
def bert_score(s1,s2):
  p = !bert-score --lang en -r s1 -c s2 --rescale_with_baseline
  p1 = p[len(p)-1].split()
  return float(p1[4])

In [ ]:
bert_score(s1,s2)

0.864428

# spacy similarity

In [ ]:
import spacy
!python -m spacy download en_core_web_lg 


     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=6dff758fa861c084e37ec7f42afd7f19d3f3db42684662067d23b0bc7429e06d
  Stored in directory: /tmp/pip-ephem-wheel-cache-mbmxokiq/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def spacy_similar(text1, text2):
    base = nlp(process_text(text1))
    compare = nlp(process_text(text2))
    return base.similarity(compare)

# quora dataset

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/quora dataset/train.csv")

In [ ]:
dataset[6:9]

,id,qid1,qid2,question1,question2,is_duplicate
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0


# checking the accuracy of each similarity scores

In [ ]:
cosine_list = []

In [ ]:
dice_list = []

In [ ]:
bert_list = []

In [ ]:
spacy_list = []

In [ ]:
len(dataset)

404290

In [ ]:
for i in range(3000):
  s1 = dataset["question1"][i]
  s2 = dataset["question2"][i]
  score = cosine_similarity(s1,s2)
  if(score >= 0.75):
    cosine_list.append(1)
  else:
    cosine_list.append(0)


In [ ]:
for i in range(3000):
  s1 = dataset["question1"][i]
  s2 = dataset["question2"][i]
  score = dice_coefficient(s1,s2)
  if(score >= 0.75):
    dice_list.append(1)
  else:
    dice_list.append(0)


In [ ]:
for i in range(3000):
  s1 = dataset["question1"][i]
  s2 = dataset["question2"][i]
  score = spacy_similar(s1,s2)
  if(score >= 0.75):
    spacy_list.append(1)
  else:
    spacy_list.append(0)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [ ]:
for i in range(3000):
  s1 = dataset["question1"][i]
  s2 = dataset["question2"][i]
  score = bert_score(s1,s2)
  if(score >= 0.75):
    bert_list.append(1)
  else:
    bert_list.append(0)


# calculating accuracy of each measures using confusion matrix

In [ ]:
from sklearn.metrics import f1_score
f1_score(cosine_list,dataset["is_duplicate"][0:3000])

0.4544976328248291

In [ ]:
f1_score(dice_list,dataset["is_duplicate"][0:3000])

0.5853932584269663

In [ ]:
f1_score(spacy_list,dataset["is_duplicate"][0:3000])

0.6179095882522315

In [ ]:
f1_score(bert_list,dataset["is_duplicate"][0:3000])

0.5185185185185185